In [7]:
import pandas as pd

# Load dataset
df = pd.read_csv("final_weather.csv")
df['year'] = df['year'].astype(int)
df['month'] = df['month'].astype(int)

# Assign season and season_year (separating Winter and Rabi)
def assign_season(row):
    y, m = row['year'], row['month']
    
    if m in [11, 12]:  # Winter start
        return pd.Series({'season': 'Winter', 'season_year': y + 1})
    elif m == 1:
        return pd.Series({'season': 'Winter', 'season_year': y})
    elif m in [2, 3]:
        return pd.Series({'season': 'Rabi', 'season_year': y})
    elif m in [4, 5]:
        return pd.Series({'season': 'Summer', 'season_year': y})
    elif m in [6, 7, 8]:
        return pd.Series({'season': 'Kharif', 'season_year': y})
    elif m in [9, 10]:
        return pd.Series({'season': 'Autumn', 'season_year': y})
    else:
        return pd.Series({'season': None, 'season_year': None})

df[['season', 'season_year']] = df.apply(assign_season, axis=1)

# Whole Year: Nov of previous year to Oct of current year
df['whole_year'] = df.apply(
    lambda row: row['year'] if row['month'] in [11, 12] else row['year'] - 1,
    axis=1
)

whole_df = df.groupby(['state', 'district', 'whole_year']).agg({
    'temperature_2m_mean': 'mean',
    'precipitation_sum':'mean',
    'relative_humidity_2m_mean':'mean',
    'wind_speed_10m_mean':'mean',
    'latitude': 'first',
    'longitude': 'first'
}).reset_index().rename(columns={'whole_year': 'season_year'})
whole_df['season'] = 'Whole Year'

# Seasonal aggregation
seasonal_df = df.groupby(['state', 'district', 'season_year', 'season']).agg({
    'temperature_2m_mean': 'mean',
    'precipitation_sum':'mean',
    'relative_humidity_2m_mean':'mean',
    'wind_speed_10m_mean':'mean',
    'latitude': 'first',
    'longitude': 'first'
}).reset_index()

# Merge all
final_df = pd.concat([seasonal_df, whole_df], ignore_index=True)
final_df = final_df.sort_values(by=['state', 'district', 'season_year', 'season']).reset_index(drop=True)

# Preview
final_df


,state,district,season_year,season,temperature_2m_mean,precipitation_sum,relative_humidity_2m_mean,wind_speed_10m_mean,latitude,longitude
0,Andhra Pradesh,ANANTAPUR,1999,Whole Year,26.878207,2.140012,61.161598,13.016970,14.724220,77.430674
1,Andhra Pradesh,ANANTAPUR,2000,Autumn,25.463495,3.328387,74.602151,9.585108,14.724220,77.430674
2,Andhra Pradesh,ANANTAPUR,2000,Kharif,25.849892,3.307706,72.830108,19.102079,14.724220,77.430674
3,Andhra Pradesh,ANANTAPUR,2000,Rabi,27.166518,1.555061,46.659066,9.001947,14.724220,77.430674
4,Andhra Pradesh,ANANTAPUR,2000,Summer,30.824892,0.855054,48.850000,12.573710,14.724220,77.430674
...,...,...,...,...,...,...,...,...,...,...
45877,West Bengal,PURULIA,2011,Rabi,23.972638,0.429032,48.365783,7.387212,23.202097,86.322899
45878,West Bengal,PURULIA,2011,Summer,29.388280,2.743602,60.327419,7.153280,23.202097,86.322899
45879,West Bengal,PURULIA,2011,Whole Year,19.547419,0.015000,69.427957,5.885645,23.202097,86.322899
45880,West Bengal,PURULIA,2011,Winter,19.290538,0.585950,64.475627,6.525448,23.202097,86.322899


In [8]:
agriculture=pd.read_csv('updated_crop.csv')

In [9]:
agriculture

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andhra Pradesh,ANANTAPUR,2001,Kharif,Arhar/Tur,27942.0,7628.0
1,Andhra Pradesh,ANANTAPUR,2001,Kharif,Bajra,1595.0,526.0
2,Andhra Pradesh,ANANTAPUR,2001,Kharif,Castor seed,3191.0,900.0
3,Andhra Pradesh,ANANTAPUR,2001,Kharif,Cotton(lint),8552.0,9055.0
4,Andhra Pradesh,ANANTAPUR,2001,Kharif,Dry chillies,3765.0,7127.0
...,...,...,...,...,...,...,...
105453,West Bengal,PURULIA,2010,Rabi,Wheat,2013.0,5152.0
105454,West Bengal,PURULIA,2010,Summer,Maize,258.0,391.0
105455,West Bengal,PURULIA,2010,Summer,Rice,105.0,281.0
105456,West Bengal,PURULIA,2010,Whole Year,Sugarcane,303.0,11541.0


In [16]:
agriculture.isnull().sum()

State_Name          0
District_Name       0
Crop_Year           0
Season              0
Crop                0
Area                0
Production       2438
dtype: int64

In [11]:
agriculture.columns

Index(['State_Name', 'District_Name', 'Crop_Year', 'Season', 'Crop', 'Area',
       'Production'],
      dtype='object')

In [12]:
final_df.columns

Index(['state', 'district', 'season_year', 'season', 'temperature_2m_mean',
       'precipitation_sum', 'relative_humidity_2m_mean', 'wind_speed_10m_mean',
       'latitude', 'longitude'],
      dtype='object')

In [13]:
import pandas as pd



# Ensure consistent column naming and case
agriculture['State_Name'] =agriculture['State_Name'].str.strip().str.lower()
agriculture['District_Name'] = agriculture['District_Name'].str.strip().str.lower()
agriculture['Season'] = agriculture['Season'].str.strip().str.lower()

final_df['state'] = final_df['state'].str.strip().str.lower()
final_df['district'] = final_df['district'].str.strip().str.lower()
final_df['season'] = final_df['season'].str.strip().str.lower()

# Merge on the matching keys
merged_df = pd.merge(
    agriculture,
    final_df,
    left_on=['State_Name', 'District_Name', 'Crop_Year', 'Season'],
    right_on=['state', 'district', 'season_year', 'season'],
    how='left'
)

# Drop extra matching columns (state, district, etc.)
merged_df = merged_df.drop(columns=['state', 'district', 'season_year', 'season'])

# Optional: reorder or check columns
print(merged_df.columns)
merged_df.head()


Index(['State_Name', 'District_Name', 'Crop_Year', 'Season', 'Crop', 'Area',
       'Production', 'temperature_2m_mean', 'precipitation_sum',
       'relative_humidity_2m_mean', 'wind_speed_10m_mean', 'latitude',
       'longitude'],
      dtype='object')


,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production,temperature_2m_mean,precipitation_sum,relative_humidity_2m_mean,wind_speed_10m_mean,latitude,longitude
0,andhra pradesh,anantapur,2001,kharif,Arhar/Tur,27942.0,7628.0,27.047742,1.125806,65.310753,21.16509,14.72422,77.430674
1,andhra pradesh,anantapur,2001,kharif,Bajra,1595.0,526.0,27.047742,1.125806,65.310753,21.16509,14.72422,77.430674
2,andhra pradesh,anantapur,2001,kharif,Castor seed,3191.0,900.0,27.047742,1.125806,65.310753,21.16509,14.72422,77.430674
3,andhra pradesh,anantapur,2001,kharif,Cotton(lint),8552.0,9055.0,27.047742,1.125806,65.310753,21.16509,14.72422,77.430674
4,andhra pradesh,anantapur,2001,kharif,Dry chillies,3765.0,7127.0,27.047742,1.125806,65.310753,21.16509,14.72422,77.430674


In [19]:
merged_df=merged_df.dropna()

In [20]:
merged_df

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production,temperature_2m_mean,precipitation_sum,relative_humidity_2m_mean,wind_speed_10m_mean,latitude,longitude
0,andhra pradesh,anantapur,2001,kharif,Arhar/Tur,27942.0,7628.0,27.047742,1.125806,65.310753,21.165090,14.724220,77.430674
1,andhra pradesh,anantapur,2001,kharif,Bajra,1595.0,526.0,27.047742,1.125806,65.310753,21.165090,14.724220,77.430674
2,andhra pradesh,anantapur,2001,kharif,Castor seed,3191.0,900.0,27.047742,1.125806,65.310753,21.165090,14.724220,77.430674
3,andhra pradesh,anantapur,2001,kharif,Cotton(lint),8552.0,9055.0,27.047742,1.125806,65.310753,21.165090,14.724220,77.430674
4,andhra pradesh,anantapur,2001,kharif,Dry chillies,3765.0,7127.0,27.047742,1.125806,65.310753,21.165090,14.724220,77.430674
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105453,west bengal,purulia,2010,rabi,Wheat,2013.0,5152.0,25.307661,0.283180,46.610599,6.913306,23.202097,86.322899
105454,west bengal,purulia,2010,summer,Maize,258.0,391.0,32.531505,1.117742,47.820430,8.522419,23.202097,86.322899
105455,west bengal,purulia,2010,summer,Rice,105.0,281.0,32.531505,1.117742,47.820430,8.522419,23.202097,86.322899
105456,west bengal,purulia,2010,whole year,Sugarcane,303.0,11541.0,24.805404,4.440341,68.917595,7.303917,23.202097,86.322899


In [21]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102797 entries, 0 to 105457
Data columns (total 13 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   State_Name                 102797 non-null  object 
 1   District_Name              102797 non-null  object 
 2   Crop_Year                  102797 non-null  int64  
 3   Season                     102797 non-null  object 
 4   Crop                       102797 non-null  object 
 5   Area                       102797 non-null  float64
 6   Production                 102797 non-null  float64
 7   temperature_2m_mean        102797 non-null  float64
 8   precipitation_sum          102797 non-null  float64
 9   relative_humidity_2m_mean  102797 non-null  float64
 10  wind_speed_10m_mean        102797 non-null  float64
 11  latitude                   102797 non-null  float64
 12  longitude                  102797 non-null  float64
dtypes: float64(8), int64(1), object(4)

In [22]:
len(merged_df['District_Name'].unique())

616

In [25]:
merged_df.to_csv('project/data_set_of_2001-10.csv',index=False)